In [ ]:
from langgraph.graph import StateGraph, START,END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import TypedDict

SyntaxError: invalid syntax (1115326090.py, line 2)

In [18]:
load_dotenv()
model = ChatOpenAI()

In [53]:
class CricterState(TypedDict, total=False):
    # Input
    runs: int
    balls: int
    four: int
    six: int

    # Computed
    strike_rate: float
    ball_per_boundary: float
    boundry_percentage: float
    summary: str


In [55]:
def calculateStrikeRate(state: CricterState) -> CricterState:
    runs = state['runs']
    balls = state['balls']
    state['strike_rate'] = round((runs / balls) * 100, 2) if balls > 0 else 0
    return state


def calculateBallPerBoundary(state: CricterState) -> CricterState:
    balls = state['balls']
    boundaries = state['four'] + state['six']
    state['ball_per_boundary'] = round(balls / boundaries, 2) if boundaries > 0 else None
    return state



def calculateBoundaryPercentage(state: CricterState) -> CricterState:
    runs = state['runs']
    boundary_runs = (4 * state['four']) + (6 * state['six'])
    state['boundry_percentage'] = round((boundary_runs / runs) * 100, 2) if runs > 0 else 0
    return state


def  summary(state: CricterState)-> CricterState:
    summary = f"""
        Strike Rate: {state['strike_rate']}\n\n
        Ball per Boundary: {state['ball_per_boundary']}\n\n
        Boundary per: {state['boundry_percentage']}\n\n"""
    state['summary'] = summary 
    return {'summary': state['summary'] }

In [57]:
graph = StateGraph(CricterState)

graph.add_node("strike_rate", calculateStrikeRate)
graph.add_node("ball_per_boundary", calculateBallPerBoundary)
graph.add_node("boundary_percentage", calculateBoundaryPercentage)
graph.add_node("summary", summary)

graph.add_edge(START, "strike_rate")
graph.add_edge("strike_rate", "ball_per_boundary")
graph.add_edge("ball_per_boundary", "boundary_percentage")
graph.add_edge("boundary_percentage", "summary")
graph.add_edge("summary", END)

workflow = graph.compile()


In [60]:
initial_state = {
    'runs':100,
    'balls':50,
    'six':5,
    'four':5
}

workflow.invoke(initial_state)

{'runs': 100,
 'balls': 50,
 'four': 5,
 'six': 5,
 'strike_rate': 200.0,
 'ball_per_boundary': 5.0,
 'boundry_percentage': 50.0,
 'summary': '\n        Strike Rate: 200.0\n\n\n        Ball per Boundary: 5.0\n\n\n        Boundary per: 50.0\n\n'}